In [1]:
import sugartensor as tf
from data import SpeechCorpus, voca_size
from model import *
import numpy as np
import struct

In [2]:
# set log level to debug
tf.sg_verbosity(10)


#
# hyper parameters
#

batch_size = 16   # total batch size

#
# inputs
#

# corpus input tensor
data = SpeechCorpus(batch_size=batch_size * tf.sg_gpus())

I 1209:12:43:00.534:data.py:152] TRAIN set loaded.(total data=43658, total batch=2728)


[<tf.Tensor 'input_producer/Gather:0' shape=() dtype=string>, <tf.Tensor 'input_producer/Gather_1:0' shape=() dtype=string>]


In [3]:
# parallel loss tower
@tf.sg_parallel
def get_loss(opt):
    # encode audio feature
    logit = get_logit(opt.input[opt.gpu_index], voca_size=voca_size)
    # CTC loss
    return logit.sg_ctc(target=opt.target[opt.gpu_index], seq_len=opt.seq_len[opt.gpu_index])


In [4]:
# mfcc feature of audio
inputs = tf.split(data.mfcc, tf.sg_gpus(), axis=0)
# target sentence label
labels = tf.split(data.label, tf.sg_gpus(), axis=0)

In [ ]:
inputs

In [ ]:
labels

In [5]:
seq_len = []
for input_ in inputs:
    seq_len.append(tf.not_equal(input_.sg_sum(axis=2), 0.).sg_int().sg_sum(axis=1))


In [ ]:
seq_len

In [6]:
tf.sg_train(lr=0.0001, loss=get_loss(input=inputs, target=labels, seq_len=seq_len),
            ep_size=data.num_batch, max_ep=50)

I 1209:12:43:34.144:sg_train.py:327] Training started from epoch[000]-step[0].


INFO:tensorflow:global_step/sec: 0


train:   0%|                        | 7/2728 [00:52<5:42:27,  7.55s/b]

INFO:tensorflow:global_step/sec: 0.118432


train:   0%|                       | 12/2728 [01:57<9:09:36, 12.14s/b]

...

INFO:tensorflow:global_step/sec: 0.0833315


train:   2%|▍                      | 47/2728 [08:56<7:43:52, 10.38s/b]

INFO:tensorflow:global_step/sec: 0.0999985


train:  16%|███▏                | 433/2728 [1:18:43<8:11:27, 12.85s/b]

KeyboardInterrupt: 

In [ ]:
mfcc = np.load('asset/data/preprocess/mfcc/p376_295.wav.npy', allow_pickle=False)

In [ ]:
mfcc.shape

In [8]:
%tensorboard --logdir asset/train/log

UsageError: Line magic function `%tensorboard` not found.
